# vLLM PagedAttention 완벽 분해

**PagedAttention**은 사실 딥러닝 기술이라기보다는 **운영체제(OS)의 메모리 관리 기법**에 가깝습니다. 실제 CUDA 커널 레벨의 코드는 너무 복잡하므로, 파이썬으로 **"PagedAttention 시뮬레이터"**를 만들어서 그 원리를 완벽하게 분해해보겠습니다.

이 코드를 실행하고 나면, 머릿속에 vLLM의 메모리 구조가 그림처럼 그려질 겁니다.

## 1교시: 개념 잡기 (바인더 노트 비유)

코드를 짜기 전에 딱 하나만 기억합시다.

* **기존 방식:** 스프링 노트입니다. 페이지를 찢거나 중간에 끼워 넣을 수 없습니다. 10장을 쓸지 모르고 100장짜리 노트를 사면 90장은 버리는 겁니다. (연속된 메모리 필수)
* **PagedAttention:** **'바인더(Loose-leaf) 노트'**입니다. 낱장(Block) 단위로 종이를 꺼내서 아무 데나 철해두고, **'목차(Block Table)'**에 순서만 적어두면 됩니다. 종이 낭비가 없죠.

## 2교시: PagedAttention 시뮬레이터 구현

이 코드는 GPU 메모리 할당 과정을 흉내 냅니다.

* **Logical Memory:** 사용자가 보는 문장 (연속됨)
* **Physical Memory:** 실제 데이터가 저장되는 곳 (뒤죽박죽 섞임)
* **Block Table:** 이 둘을 연결해 주는 지도

In [ ]:
import random
from typing import List, Dict, Optional

class PhysicalBlock:
    """
    물리적인 메모리 블록 (GPU의 작은 조각)
    여기서는 편의상 하나의 블록에 4개의 토큰을 저장한다고 가정합니다 (BLOCK_SIZE = 4).
    """
    def __init__(self, block_id: int, block_size: int = 4):
        self.block_id = block_id
        self.size = block_size
        self.data = [None] * block_size  # 처음엔 비어있음 [None, None, None, None]
        self.filled_count = 0

    def append(self, token: str) -> bool:
        if self.filled_count < self.size:
            self.data[self.filled_count] = token
            self.filled_count += 1
            return True  # 저장 성공
        return False  # 꽉 차서 실패

    def __repr__(self):
        return f"[Block {self.block_id}]: {self.data}"

class VLLMMemoryManager:
    """
    vLLM의 핵심인 메모리 관리자 (OS 역할)
    """
    def __init__(self, total_blocks: int = 16, block_size: int = 4):
        self.block_size = block_size
        # 1. 물리적 메모리 공간 생성 (GPU VRAM)
        self.physical_memory = [PhysicalBlock(i, block_size) for i in range(total_blocks)]
        
        # 2. 사용 가능한 블록 리스트 (Free List)
        # 실제로는 물리적으로 떨어져 있어도 상관없음을 보여주기 위해 섞습니다.
        self.free_blocks = list(range(total_blocks))
        random.shuffle(self.free_blocks) 
        
        # 3. 요청별 블록 매핑 테이블 (Request ID -> List[Physical Block IDs])
        self.block_tables: Dict[str, List[int]] = {}

    def allocate_block(self) -> Optional[int]:
        """빈 물리 블록 하나를 가져옵니다."""
        if not self.free_blocks:
            return None # OOM (Out of Memory)
        return self.free_blocks.pop()

    def append_token(self, request_id: str, token: str):
        """
        핵심 로직: 토큰을 생성해서 KV Cache에 추가하는 과정
        """
        # 1. 해당 요청의 블록 테이블이 없으면 생성
        if request_id not in self.block_tables:
            print(f"--- [New Request] '{request_id}' 시작 ---")
            self.block_tables[request_id] = []

        # 2. 현재 사용 중인 마지막 물리 블록 확인
        current_block_id = None
        if self.block_tables[request_id]:
            current_block_id = self.block_tables[request_id][-1]
        
        # 3. 블록이 없거나 꽉 찼으면, 새로운 물리 블록 할당 (PagedAttention의 핵심!)
        if current_block_id is None or \
           self.physical_memory[current_block_id].filled_count >= self.block_size:
            
            new_block_id = self.allocate_block()
            if new_block_id is None:
                raise Exception("GPU Memory Full!")
            
            self.block_tables[request_id].append(new_block_id)
            current_block_id = new_block_id
            print(f"👉 [Alloc] 새로운 물리 블록 {current_block_id}번 할당됨")

        # 4. 물리 블록에 데이터 저장
        self.physical_memory[current_block_id].append(token)
        print(f"   [Write] 토큰 '{token}' -> 물리 블록 {current_block_id}번에 저장")

    def print_state(self, request_id: str):
        """현재 메모리 상태를 시각화해서 보여줍니다."""
        print(f"\n📊 [{request_id}] 의 PagedAttention 상태")
        table = self.block_tables.get(request_id, [])
        
        # 1. Logical View (사용자가 보는 문장)
        logical_text = []
        for block_id in table:
            block = self.physical_memory[block_id]
            logical_text.extend([t for t in block.data if t is not None])
        print(f"  1) 논리적 뷰 (Logical): {logical_text}")

        # 2. Block Table (매핑 정보)
        print(f"  2) 블록 테이블 (Mapping): {table} (논리적 순서 0, 1.. -> 물리적 주소 {table})")

        # 3. Physical View (실제 저장 위치)
        print(f"  3) 물리적 뷰 (Physical):")
        for block_id in table:
            print(f"     {self.physical_memory[block_id]}")
        print("-" * 50)

In [ ]:
# ==========================================
# 실행 시뮬레이션
# ==========================================

# 1. 매니저 초기화 (Block Size = 4)
vllm = VLLMMemoryManager(total_blocks=10, block_size=4)

# 2. 문장 생성 시뮬레이션 ("Deep Learning is fun")
# 마치 LLM이 한 토큰씩 생성하는 상황입니다.
req_id = "User_A"
tokens = ["Deep", "Learn", "ing", "is", "very", "fun", "to", "study"]

for token in tokens:
    vllm.append_token(req_id, token)

# 3. 최종 상태 확인
vllm.print_state(req_id)

## 3교시: 코드 실행 결과 해석 (강의)

위 코드를 실행하면 아래와 비슷한 결과가 나옵니다. (블록 ID는 랜덤이므로 실행할 때마다 달라집니다.)

### [시뮬레이션 로그 분석]

```text
--- [New Request] 'User_A' 시작 ---
👉 [Alloc] 새로운 물리 블록 7번 할당됨
   [Write] 토큰 'Deep' -> 물리 블록 7번에 저장
   [Write] 토큰 'Learn' -> 물리 블록 7번에 저장
   [Write] 토큰 'ing' -> 물리 블록 7번에 저장
   [Write] 토큰 'is' -> 물리 블록 7번에 저장
👉 [Alloc] 새로운 물리 블록 2번 할당됨  <-- 중요! 7번이 꽉 차니까 2번을 가져옴
   [Write] 토큰 'very' -> 물리 블록 2번에 저장
   ...
```

### [최종 상태 시각화]

```text
📊 [User_A] 의 PagedAttention 상태
  1) 논리적 뷰 (Logical): ['Deep', 'Learn', 'ing', 'is', 'very', 'fun', 'to', 'study']
  2) 블록 테이블 (Mapping): [7, 2] 
  3) 물리적 뷰 (Physical):
     [Block 7]: ['Deep', 'Learn', 'ing', 'is']
     [Block 2]: ['very', 'fun', 'to', 'study']
```

### [핵심 포인트 3가지]

#### 1. **비연속성 (Non-contiguous):**
* 논리적으로는 `'is'` 다음에 `'very'`가 옵니다.
* 하지만 물리적으로는 `Block 7` 다음에 `Block 8`이 아니라 뜬금없는 `Block 2`에 저장되었습니다.
* **의의:** 메모리의 빈 공간이 어디에 있든 상관없이 잡아다 쓸 수 있습니다. (메모리 단편화 해결)

#### 2. **동적 할당 (On-demand Allocation):**
* 처음부터 메모리를 다 잡지 않았습니다. `Block 7`이 꽉 찰 때(4개 토큰), 비로소 `Block 2`를 할당했습니다.
* **의의:** 만약 문장이 `'Deep Learning is'`에서 끝났다면? `Block 2`는 아예 할당되지 않았을 겁니다. 미리 잡아둔 공간의 낭비(Reserved slots waste)가 'Zero'가 됩니다.

#### 3. **블록 테이블 (The Map):**
* vLLM은 `Block Table: [7, 2]` 라는 정보만 딱 쥐고 있습니다.
* 나중에 "5번째 토큰(`very`)을 가져와라" 하면 어떻게 계산할까요?
  * Block Size = 4 이므로,
  * `5 // 4 = 1` (인덱스 1번 블록 -> **Physical Block 2**)
  * `5 % 4 = 1` (블록 내 1번째 오프셋 -> **'very'**)
* 이 계산이 매우 빠르기 때문에 성능 저하가 없는 것입니다.

---

이 코드를 통해 PagedAttention이 어떻게 GPU 메모리를 '테트리스' 하듯이 빈틈없이 채워 넣는지 이해하셨나요? 이것이 vLLM이 다른 라이브러리보다 압도적으로 빠르고 효율적인 이유입니다.

---

# KV Cache: 책장에 꽂히는 책 그 자체

**PagedAttention**이 '책을 꽂을 **책장(Memory)** 관리법'이었다면, **KV Cache**는 그 책장에 꽂히는 **'책(Data) 그 자체'**입니다.

vLLM을 이해하려면, 도대체 왜 **"캐싱(Caching)"**을 안 하면 LLM이 느려터지게 되는지, 그리고 KV Cache가 수학적으로 어떤 마법을 부리는지 직접 눈으로 확인해야 합니다.

이번 시간엔 **KV Cache**의 원리를 씹어 먹을 수 있게 파이썬 시뮬레이션으로 보여드리겠습니다.

## 4교시: 직관적 이해 (일기장 비유)

여러분이 **"나는 어제 학교에 가서 밥을 먹었다"** 라는 문장을 쓴다고 가정해봅시다.

### 1. KV Cache가 없는 경우 (치매 걸린 작가)

* "나는" (씀)
* (기억 리셋) "나는" + "어제" (다시 읽고 씀)
* (기억 리셋) "나는 어제" + "학교에" (다시 읽고 씀)
* (기억 리셋) "나는 어제 학교에" + "가서" ...
* **문제점:** 단어 하나 쓸 때마다 처음부터 끝까지 다시 다 읽고 해석해야 합니다. 문장이 길어질수록 기하급수적으로 느려집니다.

### 2. KV Cache가 있는 경우 (스마트한 작가)

* "나는" (씀. '나는'에 대한 정보 포스트잇에 메모)
* "어제" (씀. 아까 메모한거 쓱 보고, '어제' 정보 메모)
* "학교에" (씀. 아까 메모들 쓱 보고, '학교에' 정보 메모)
* **핵심:** 과거의 단어들을 다시 계산하지 않고, 미리 저장해둔 **메모(Key, Value)**만 참조합니다.

## 5교시: KV Cache 성능 비교 시뮬레이터

말로만 하면 안 와닿습니다. 코드로 **"연산 횟수"**를 카운팅해서 얼마나 차이가 나는지 보여드리겠습니다.

이 코드는 실제 행렬 연산 대신, **'연산을 몇 번 수행했는가(Cost)'**를 시뮬레이션합니다.

In [ ]:
import time

class ModelSimulator:
    def __init__(self):
        self.op_count = 0

    def compute_token_representation(self, token):
        """
        토큰 하나를 숫자로 바꾸고(Embedding), Q, K, V 벡터를 만드는 과정
        엄청난 고비용 연산이라고 가정합니다.
        """
        self.op_count += 1
        return f"Vector({token})"

    def attention_calculation(self, current_token, past_vectors):
        """
        현재 토큰과 과거의 모든 토큰 간의 관계를 계산 (Attention)
        """
        self.op_count += len(past_vectors)

In [ ]:
# ==========================================
# 시나리오 1: KV Cache 없이 생성 (No Cache)
# ==========================================
print("🚫 [Scenario 1] KV Cache 미사용 (매번 처음부터 다시 계산)")
model_no_cache = ModelSimulator()
generated_tokens = []
input_prompt = ["Deep", "Learning", "is", "Fun"]

for step in range(5):
    current_context = input_prompt + generated_tokens
    
    vectors = []
    for token in current_context:
        vec = model_no_cache.compute_token_representation(token)
        vectors.append(vec)
    
    model_no_cache.attention_calculation("New_Token", vectors)
    
    generated_tokens.append(f"Gen_{step}")
    print(f"  Step {step+1}: 현재 문장 길이 {len(current_context)} -> 누적 연산 횟수: {model_no_cache.op_count}")

final_cost_no_cache = model_no_cache.op_count

In [ ]:
# ==========================================
# 시나리오 2: KV Cache 사용 (With Cache)
# ==========================================
print("\n✅ [Scenario 2] KV Cache 사용 (과거는 저장해두고 재사용)")
model_with_cache = ModelSimulator()
generated_tokens = []
kv_cache = []

print("  [Prefill] 초기 프롬프트 처리 중...")
for token in input_prompt:
    vec = model_with_cache.compute_token_representation(token)
    kv_cache.append(vec)

for step in range(5):
    new_token_input = f"Gen_{step-1}" if step > 0 else input_prompt[-1]
    
    current_vec = model_with_cache.compute_token_representation("New_Token")
    
    model_with_cache.attention_calculation("New_Token", kv_cache)
    
    kv_cache.append(current_vec)
    generated_tokens.append(f"Gen_{step}")
    
    print(f"  Step {step+1}: 캐시 크기 {len(kv_cache)} -> 누적 연산 횟수: {model_with_cache.op_count}")

final_cost_with_cache = model_with_cache.op_count

In [ ]:
# ==========================================
# 최종 결과 비교
# ==========================================
print("\n" + "="*40)
print(f"📊 최종 연산 비용 비교 (낮을수록 빠름)")
print(f"1. No Cache Cost  : {final_cost_no_cache} (기하급수적 증가)")
print(f"2. With Cache Cost: {final_cost_with_cache} (선형적 증가)")
print(f"🚀 효율 향상: 약 {final_cost_no_cache / final_cost_with_cache:.1f}배 더 적은 연산 수행")
print("="*40)

## 6교시: 결과 해석 및 기술 면접 대비

코드를 실행해보면 **KV Cache**가 있고 없고의 차이가 엄청나다는 것을 알 수 있습니다.

### 1. 왜 Q(Query)는 캐싱 안 하나요?

* 학생들이 자주 묻는 질문입니다.
* **Query**는 "지금 내가 궁금한 것(현재 토큰)"입니다. 매 스텝마다 질문자가 바뀌기 때문에 캐싱할 수 없습니다.
* **Key, Value**는 "과거의 기억"입니다. "나는" 이라는 단어가 과거에 있었다는 사실(Key)과 그 의미(Value)는 변하지 않으므로 저장(Cache)해두는 겁니다.

### 2. 메모리(VRAM)와의 전쟁

* 시뮬레이션에서 `kv_cache` 리스트가 계속 길어지는 게 보이시나요?
* **속도(Compute)**는 빨라졌지만, 그 대가로 **메모리(Memory)**를 엄청나게 잡아먹습니다.
* 문장이 길어지면 이 `kv_cache` 리스트가 너무 커져서 GPU 메모리가 터집니다(OOM).
* **바로 여기서 vLLM의 PagedAttention이 등장하는 겁니다!**
* "어차피 커질 `kv_cache`라면, 빈 공간 없이 쪼개서(Blocking) 효율적으로 저장하자."

### 3. 요약 (이것만 기억하세요)

* **No Cache:** O(N²) 비용 발생. 문장이 길어지면 컴퓨터 멈춤.
* **KV Cache:** O(N) 비용. 대신 메모리를 많이 씀. (메모리 ↔ 속도 교환)
* **vLLM:** KV Cache를 OS처럼 관리해서 메모리 낭비를 줄이고, 더 많은 KV Cache를 쑤셔 넣음.

---

이제 **KV Cache**가 무엇이고, 왜 **vLLM(PagedAttention)**이 이 캐시를 관리하는 데 목숨을 거는지 이해가 되셨나요? 이 개념들이 연결되어야 진짜 엔지니어입니다.